In [1]:
import time
from sklearn import linear_model
from sklearn import svm
from sklearn import ensemble
import xgboost
from keras.layers import *
from keras import layers, models, optimizers
from keras.preprocessing import text, sequence
from sklearn import preprocessing
from sklearn import naive_bayes
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import drive
import pickle

Using TensorFlow backend.


In [2]:
# Load data
drive.mount('/content/drive', force_remount=True)
DATA_PATH = "/content/drive/My Drive/NLP"

x_data = pickle.load(open(DATA_PATH + '/Processed Data/x_data.pkl', 'rb'))
y_data = pickle.load(open(DATA_PATH + '/Processed Data/y_data.pkl', 'rb'))

x_test = pickle.load(open(DATA_PATH + '/Processed Data/x_test.pkl', 'rb'))
y_test = pickle.load(open(DATA_PATH + '/Processed Data/y_test.pkl', 'rb'))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# Transform data
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(x_data)

x_data_tfidf = tfidf_vect_ngram.transform(x_data)
x_test_tfidf = tfidf_vect_ngram.transform(x_test)

In [4]:
# SVD
svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(x_data_tfidf)

x_data_tfidf_svd = svd.transform(x_data_tfidf)
x_test_tfidf_svd = svd.transform(x_test_tfidf)

In [5]:
# Train Model
def train_model(classifier, x_data, y_data, x_test, y_test, is_neuralnet=False, n_epochs=100):
    x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=42)

    if is_neuralnet:
        classifier.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=n_epochs, batch_size=512)

        val_predictions = classifier.predict(x_val)
        test_predictions = classifier.predict(x_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(x_train, y_train)

        train_predictions = classifier.predict(x_train)
        val_predictions = classifier.predict(x_val)
        test_predictions = classifier.predict(x_test)

    # Evaluation

    print("> Validation Accuracy: ", metrics.accuracy_score(val_predictions, y_val))

    print("> Test Accuracy: ", metrics.accuracy_score(test_predictions, y_test))

    print("> Validation Precision: ", metrics.precision_score(y_val, val_predictions, average='macro'))

    print("> Test Precision: ", metrics.precision_score(y_test, test_predictions, average='macro'))

    print("> Validation Recall: ", metrics.recall_score(y_val, val_predictions, average='macro'))

    print("> Test Recall: ", metrics.recall_score(y_test, test_predictions, average='macro'))

    print("> Validation F1 Score: ", metrics.f1_score(y_val, val_predictions, average='macro'))

    print("> Test F1 Score: ", metrics.f1_score(y_test, test_predictions, average='macro'))

In [ ]:
# Naive Bayes
# Naive Bayes (Multinomial)
print("\n\n----- Naive Bayes (Multinomial) -----")
nbm_start = time.time()
train_model(naive_bayes.MultinomialNB(), x_data_tfidf, y_data, x_test_tfidf, y_test, is_neuralnet=False)
nbm_stop = time.time()
print(f"> Training time: {nbm_stop - nbm_start}s")



----- Naive Bayes (Multinomial) -----
> Validation Accuracy:  0.8144418255838869
> Test Accuracy:  0.8267389010632253
> Validation Precision:  0.8239420745965697
> Test Precision:  0.7578213490605008
> Validation Recall:  0.8027363488793946
> Test Recall:  0.7999891282160969
> Validation F1 Score:  0.8073793433249031
> Test F1 Score:  0.7618122053679133
> Training time: 1.1583480834960938s


In [ ]:
# Naive Bayes (Bernoulli)
print("\n\n----- Naive Bayes (Bernoulli) -----")
nbb_start = time.time()
train_model(naive_bayes.BernoulliNB(), x_data_tfidf, y_data, x_test_tfidf, y_test, is_neuralnet=False)
nbb_stop = time.time()
print(f"> Training time: {nbb_stop - nbb_start}s")



----- Naive Bayes (Bernoulli) -----
> Validation Accuracy:  0.7668737947289479
> Test Accuracy:  0.7878621865108352
> Validation Precision:  0.7699038486756739
> Test Precision:  0.7204126299102829
> Validation Recall:  0.7612265834966994
> Test Recall:  0.778069688289809
> Validation F1 Score:  0.7600874679563089
> Test F1 Score:  0.7312551244443752
> Training time: 1.4384613037109375s


In [ ]:
# Logistic Regression
print("\n\n----- Logistic Regression -----")
lr_start = time.time()
train_model(linear_model.LogisticRegression(), x_data_tfidf, y_data, x_test_tfidf, y_test, is_neuralnet=False)
lr_stop = time.time()
print(f"> Training time: {lr_stop - lr_start}s")



----- Logistic Regression -----


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


> Validation Accuracy:  0.8737947289479323
> Test Accuracy:  0.8688418147877607
> Validation Precision:  0.8747081975276904
> Test Precision:  0.8073578452887772
> Validation Recall:  0.8647290060385876
> Test Recall:  0.8482830816395908
> Validation F1 Score:  0.8683796914742599
> Test F1 Score:  0.8194722209712961
> Training time: 46.87932586669922s


In [ ]:
# Support Vector Machine
print("\n\n----- Support Vector Machine -----")
svm_start = time.time()
train_model(svm.SVC(), x_data_tfidf_svd, y_data, x_test_tfidf_svd, y_test, is_neuralnet=False)
svm_stop = time.time()
print(f"> Training time: {svm_stop - svm_start}s")



----- Support Vector Machine -----
> Validation Accuracy:  0.8673666166702378
> Test Accuracy:  0.8604212320428537
> Validation Precision:  0.8645144910339452
> Test Precision:  0.7890784044653181
> Validation Recall:  0.8614197529989157
> Test Recall:  0.8389809886340895
> Validation F1 Score:  0.8622786288201751
> Test F1 Score:  0.8062765027541737
> Training time: 1385.8506214618683s


In [ ]:
# Random Forest Classifier
print("\n\n----- Random Forest Classifier -----")
rfc_start = time.time()
train_model(ensemble.RandomForestClassifier(), x_data_tfidf_svd, y_data, x_test_tfidf_svd, y_test, is_neuralnet=False)
rfc_stop = time.time()
print(f"> Training time: {rfc_stop - rfc_start}s")



----- Random Forest Classifier -----
> Validation Accuracy:  0.8052281979858582
> Test Accuracy:  0.8094919243567892
> Validation Precision:  0.8101302088221888
> Test Precision:  0.7331266545634075
> Validation Recall:  0.7930253867126018
> Test Recall:  0.7747364675925617
> Validation F1 Score:  0.7989855641287471
> Test F1 Score:  0.7413094946778536
> Training time: 116.58514499664307s


In [ ]:
# XGBoost
print("\n\n----- XGBoost -----")
xgb_start = time.time()
train_model(xgboost.XGBClassifier(), x_data_tfidf_svd, y_data, x_test_tfidf_svd, y_test, is_neuralnet=False)
xgb_stop = time.time()
print(f"> Training time: {xgb_stop - xgb_start}s")



----- XGBoost -----
> Validation Accuracy:  0.8035140347118063
> Test Accuracy:  0.7996712929145362
> Validation Precision:  0.8036248243813601
> Test Precision:  0.7156032885225023
> Validation Recall:  0.794790530156443
> Test Recall:  0.7710278849601445
> Validation F1 Score:  0.7972954554692699
> Test F1 Score:  0.7307782772360046
> Training time: 1073.605946779251s


In [ ]:
# Deep Neural Network
dnn_start = time.time()
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)


def create_dnn_model():
    input_layer = Input(shape=(300,))
    layer = Dense(1024, activation='relu')(input_layer)
    layer = Dense(1024, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    output_layer = Dense(13, activation='softmax')(layer)

    classifier = models.Model(input_layer, output_layer)
    classifier.compile(optimizer=optimizers.Adam(
    ), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return classifier


classifier = create_dnn_model()
print("\n\n----- Deep Neural Network -----")
train_model(classifier=classifier, x_data=x_data_tfidf_svd, y_data=y_data_n, x_test=x_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True)
dnn_stop = time.time()
print(f"> Training time: {dnn_stop - dnn_start}s")



----- Deep Neural Network -----
Train on 42001 samples, validate on 4667 samples
Epoch 1/100
42001/42001 [==============================] - 11s 250us/step - loss: 0.9761 - accuracy: 0.7134 - val_loss: 0.5237 - val_accuracy: 0.8305
Epoch 2/100
42001/42001 [==============================] - 10s 248us/step - loss: 0.4631 - accuracy: 0.8506 - val_loss: 0.4661 - val_accuracy: 0.8502
Epoch 3/100
42001/42001 [==============================] - 10s 250us/step - loss: 0.4100 - accuracy: 0.8637 - val_loss: 0.4909 - val_accuracy: 0.8408
Epoch 4/100
42001/42001 [==============================] - 11s 255us/step - loss: 0.3965 - accuracy: 0.8685 - val_loss: 0.4420 - val_accuracy: 0.8577
Epoch 5/100
42001/42001 [==============================] - 10s 248us/step - loss: 0.3527 - accuracy: 0.8823 - val_loss: 0.4288 - val_accuracy: 0.8624
Epoch 6/100
42001/42001 [==============================] - 10s 248us/step - loss: 0.3298 - accuracy: 0.8891 - val_loss: 0.4338 - val_accuracy: 0.8635
Epoch 7/100
42001

In [ ]:
# Recurrent Neural Network - LSTM
def create_lstm_model():
    input_layer = Input(shape=(300,))

    layer = Reshape((10, 30))(input_layer)
    layer = LSTM(128, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)

    output_layer = Dense(13, activation='softmax')(layer)

    classifier = models.Model(input_layer, output_layer)

    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return classifier

lstm_start = time.time()
classifier = create_lstm_model()
print("\n\n----- Recurrent Neural Network - LSTM -----")
train_model(classifier=classifier, x_data=x_data_tfidf_svd, y_data=y_data_n, x_test=x_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True)
lstm_stop = time.time()
print(f"> Training time: {lstm_stop - lstm_start}s")



----- Recurrent Neural Network - LSTM -----
Train on 42001 samples, validate on 4667 samples
Epoch 1/100
42001/42001 [==============================] - 12s 284us/step - loss: 2.3893 - accuracy: 0.1517 - val_loss: 2.2241 - val_accuracy: 0.2186
Epoch 2/100
42001/42001 [==============================] - 11s 270us/step - loss: 1.9521 - accuracy: 0.2875 - val_loss: 1.8165 - val_accuracy: 0.3349
Epoch 3/100
42001/42001 [==============================] - 11s 270us/step - loss: 1.7306 - accuracy: 0.3759 - val_loss: 1.8293 - val_accuracy: 0.3497
Epoch 4/100
42001/42001 [==============================] - 11s 270us/step - loss: 1.6170 - accuracy: 0.4277 - val_loss: 1.5785 - val_accuracy: 0.4534
Epoch 5/100
42001/42001 [==============================] - 12s 276us/step - loss: 1.5245 - accuracy: 0.4700 - val_loss: 1.4905 - val_accuracy: 0.4913
Epoch 6/100
42001/42001 [==============================] - 12s 280us/step - loss: 1.4696 - accuracy: 0.4895 - val_loss: 1.6825 - val_accuracy: 0.3756
Epoch

In [ ]:
# Recurrent Neural Network - GRU
def create_gru_model():
    input_layer = Input(shape=(300,))

    layer = Reshape((10, 30))(input_layer)
    layer = GRU(128, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)

    output_layer = Dense(13, activation='softmax')(layer)

    classifier = models.Model(input_layer, output_layer)

    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return classifier

gru_start = time.time()
classifier = create_gru_model()
print("\n\n----- Recurrent Neural Network - GRU -----")
train_model(classifier=classifier, x_data=x_data_tfidf_svd, y_data=y_data_n, x_test=x_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True, n_epochs=100)
gru_stop = time.time()
print(f"> Training time: {gru_stop - gru_start}s")



----- Recurrent Neural Network - GRU -----
Train on 42001 samples, validate on 4667 samples
Epoch 1/100
42001/42001 [==============================] - 12s 298us/step - loss: 2.5163 - accuracy: 0.1148 - val_loss: 2.3920 - val_accuracy: 0.1391
Epoch 2/100
42001/42001 [==============================] - 12s 279us/step - loss: 2.0956 - accuracy: 0.2351 - val_loss: 2.0626 - val_accuracy: 0.2366
Epoch 3/100
42001/42001 [==============================] - 12s 284us/step - loss: 1.8045 - accuracy: 0.3399 - val_loss: 1.9201 - val_accuracy: 0.2974
Epoch 4/100
42001/42001 [==============================] - 12s 283us/step - loss: 1.6826 - accuracy: 0.3886 - val_loss: 1.6284 - val_accuracy: 0.4163
Epoch 5/100
42001/42001 [==============================] - 12s 281us/step - loss: 1.5966 - accuracy: 0.4228 - val_loss: 1.9430 - val_accuracy: 0.2918
Epoch 6/100
42001/42001 [==============================] - 12s 281us/step - loss: 1.5905 - accuracy: 0.4276 - val_loss: 1.5706 - val_accuracy: 0.4414
Epoch 

In [ ]:
# Bidirectional RNN


def create_brnn_model():
    input_layer = Input(shape=(300,))

    layer = Reshape((10, 30))(input_layer)
    layer = Bidirectional(GRU(128, activation='relu'))(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)

    output_layer = Dense(13, activation='softmax')(layer)

    classifier = models.Model(input_layer, output_layer)

    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return classifier

brnn_start = time.time()
classifier = create_brnn_model()
print("\n\n----- Bidirectional RNN -----")
train_model(classifier=classifier, x_data=x_data_tfidf_svd, y_data=y_data_n, x_test=x_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True, n_epochs=100)
brnn_stop = time.time()
print(f"> Training time: {brnn_stop - brnn_start}s")



----- Bidirectional RNN -----
Train on 42001 samples, validate on 4667 samples
Epoch 1/100
42001/42001 [==============================] - 34s 812us/step - loss: 1.7735 - accuracy: 0.4332 - val_loss: 1.3326 - val_accuracy: 0.5607
Epoch 2/100
42001/42001 [==============================] - 35s 837us/step - loss: 0.8802 - accuracy: 0.7199 - val_loss: 0.7871 - val_accuracy: 0.7487
Epoch 3/100
42001/42001 [==============================] - 34s 811us/step - loss: 0.7326 - accuracy: 0.7655 - val_loss: 0.6776 - val_accuracy: 0.7814
Epoch 4/100
42001/42001 [==============================] - 38s 897us/step - loss: 0.6622 - accuracy: 0.7861 - val_loss: 0.7074 - val_accuracy: 0.7697
Epoch 5/100
42001/42001 [==============================] - 34s 818us/step - loss: 0.6170 - accuracy: 0.7996 - val_loss: 0.6246 - val_accuracy: 0.7999
Epoch 6/100
42001/42001 [==============================] - 36s 846us/step - loss: 0.5853 - accuracy: 0.8087 - val_loss: 0.6009 - val_accuracy: 0.8027
Epoch 7/100
42001/4

In [7]:
# Recurrent Convolutional Neural Network
def create_rcnn_model():
    input_layer = Input(shape=(300,))

    layer = Reshape((10, 30))(input_layer)
    layer = Bidirectional(
        GRU(128, activation='relu', return_sequences=True))(layer)
    layer = Convolution1D(100, 3, activation="relu")(layer)
    layer = Flatten()(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)

    output_layer = Dense(13, activation='softmax')(layer)

    classifier = models.Model(input_layer, output_layer)
    classifier.summary()
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return classifier

rcnn_start = time.time()
classifier = create_rcnn_model()
print("\n\n----- Recurrent Convolutional Neural Network -----")
train_model(classifier=classifier, x_data=x_data_tfidf_svd, y_data=y_data_n, x_test=x_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True, n_epochs=100)
rcnn_stop = time.time()
print(f"> Training time: {rcnn_stop - rcnn_start}s")

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 10, 30)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 10, 256)           122112    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 8, 100)            76900     
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               410112    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               2626